In [1]:
%pip install -q pyvis networkx


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path

# Define configuration constants
INTERIM_DIR = Path("../data/interim")  # Standardize path using pathlib
PROCESSED_DIR = Path("../data/processed")  # Standardize path using pathlib

gtp_graphml = INTERIM_DIR / "gtp_aurk_inhibitors.graphml"
gtp_html = PROCESSED_DIR / "gtp_aurk_inhibitors.html"


In [3]:
import xml.etree.ElementTree as ET
from pyvis.network import Network
import networkx as nx
import re

# Constants
GRAPHML_NS = {'graphml': 'http://graphml.graphdrawing.org/xmlns'}

# Function to clean HTML tags from text
def clean_html(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

# Parse the GraphML data
try:
    tree = ET.parse(gtp_graphml)
except FileNotFoundError:
    print("Error: GraphML file not found.")
    exit(1)
except ET.ParseError:
    print("Error: Invalid GraphML file.")
    exit(1)

root = tree.getroot()

# Create a NetworkX graph
G = nx.Graph()

# Extract nodes and their attributes
for node in root.findall('.//graphml:node', GRAPHML_NS):
    node_id = node.get('id')
    
    # Get node labels
    label_elem = node.find('.//graphml:data[@key="label"]', GRAPHML_NS)
    label = clean_html(label_elem.text) if label_elem is not None and label_elem.text else node_id
    
    # Get node type from labels attribute
    node_type = node.get('labels', 'Unknown')
    
    G.add_node(node_id, label=label, node_type=node_type)

# Extract edges
for edge in root.findall('.//graphml:edge', GRAPHML_NS):
    source = edge.get('source')
    target = edge.get('target')
    
    # Get edge label
    edge_label_elem = edge.find('.//graphml:data[@key="label"]', GRAPHML_NS)
    edge_label = edge_label_elem.text if edge_label_elem is not None else ''
    
    G.add_edge(source, target, label=edge_label)

# Create Pyvis network
net = Network(height="750px", width="100%", bgcolor="#ffffff", font_color="black")
net.from_nx(G)

# Customize node appearances based on type
for node in net.nodes:
    node_type = node.get('node_type', 'Unknown')
    
    if ':Target' in node_type:
        node['color'] = '#ff7f7f'  # Red for targets
        node['size'] = 25
    elif ':SyntheticOrganicLigand' in node_type:
        node['color'] = '#7f7fff'  # Blue for ligands
        node['size'] = 20
    elif ':Affinity' in node_type:
        node['color'] = '#7fff7f'  # Green for affinity
        node['size'] = 15
    else:
        node['color'] = '#cccccc'  # Gray for other nodes
        node['size'] = 10

# Customize edge appearances
for edge in net.edges:
    edge['width'] = 1
    edge['color'] = '#999999'

# Set physics options for better layout
net.set_options('''
{
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -2000,
      "centralGravity": 0.3,
      "springLength": 200,
      "springConstant": 0.04
    },
    "maxVelocity": 50,
    "minVelocity": 0.1,
    "solver": "barnesHut",
    "stabilization": {
      "enabled": true,
      "iterations": 1000
    }
  }
}
''')



In [4]:
print(gtp_html)

../data/processed/gtp_aurk_inhibitors.html


In [5]:
# Save or show the network
net.save_graph(str(gtp_html))